In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("talk")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

from ese.experiment.analysis.analyze_inf import load_cal_inference_stats
# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml results_cfg 

log:
    root: /storage/vbutoi/scratch/ESE/inference
    inference_groups: 
        - '06_04_24_WMH_DifferenceExps'

options:
    add_dice_loss_rows: True
    drop_nan_metric_rows: True 
    remove_shared_columns: False
    equal_rows_per_cfg_assert: False 

In [ ]:
inference_df = load_cal_inference_stats(
    results_cfg=results_cfg,
    load_cached=False,
)

In [ ]:
# We are limiting ourselves to looking at the calibration split for now.
inference_df = inference_df.select(split='cal').reset_index(drop=True)
# Make the calibrator column a category so that we can plot in the order we like.
inference_df['calibrator'] = inference_df['calibrator'].astype('category')
inference_df['calibrator'] = inference_df['calibrator'].cat.reorder_categories([
    'Uncalibrated',
    'TempScaling',
    'LTS',
])

In [ ]:
inference_df

In [ ]:
volume_df = inference_df.groupby([
    "calibrator",
    "data_id",
    "pretrained_seed", 
])[
    "gt_volume",
    "soft_volume",
    "hard_volume",
].sum().reset_index()

In [ ]:
volume_df.head(10)

In [ ]:
# Make two new columns, one for the soft volume error and one for the hard volume error.
volume_df['soft_volume_error'] = volume_df['soft_volume'] - volume_df['gt_volume']
volume_df['hard_volume_error'] = volume_df['hard_volume'] - volume_df['gt_volume']

## Let's see how good the soft-volume is at pure-volumetric prediction.

In [ ]:
# We need to melt our dataframe to plot it.
measure_df = pd.melt(
    volume_df,
    id_vars=["calibrator", "data_id", "pretrained_seed"],
    value_vars=["gt_volume", "soft_volume", "hard_volume"],
    var_name="volume_type",
    value_name="volume",
)

In [ ]:
g = sns.catplot(
    measure_df,
    x="calibrator",
    y="volume",
    hue="volume_type",
    col="data_id",
    col_wrap=3,
    aspect=1.5,
    sharey=False
)
# We need a sup-title to explain what we are looking at.

# Adjust the layout
plt.subplots_adjust(top=0.85)
g.fig.suptitle('Soft/Hard Volume Prediction by Calibrator', fontsize=30)

# Show the plot
plt.show()

## Another way to look at this is the distribution of errors.

In [ ]:
error_df = pd.melt(
    volume_df,
    id_vars=["calibrator", "data_id", "pretrained_seed"],
    value_vars=["soft_volume_error", "hard_volume_error"],
    var_name="volume_type",
    value_name="error",
)

In [ ]:
g = sns.catplot(
    error_df,
    x="calibrator",
    y="error",
    hue="volume_type",
    col="data_id",
    col_wrap=3,
    aspect=1.5,
    sharey=False,
)
# For each subplot make a line at y = 0 to show the error.
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='red')

# Adjust the layout
plt.subplots_adjust(top=0.85)
g.fig.suptitle('Soft/Hard Volumetric Error by Calibrator', fontsize=30)

# Show the plot
plt.show()